# 1. Libraries in Python

Python is very popular because (among other reasons) is open source and many people have developed packages based on Python. Those independent developments usually are stored separetely from the "main python" and has to be installed after installing Python. Once installed, they become a library that is called at the beginning of the algorithm using the method **import**.

### Pandas Library

"Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language."
Pandas_logo.svg

We will use pandas for reading datasets

In [ ]:
#Let's import python
import pandas as pd
#this syntax means: load the library pandas and for the rest of the code we will refer to it as pd

**NOTE**:
Observe that we are working on a virtual environment that already has installed pandas. That is why we can call it directly. In future endeavours as a programmer, you will learn how to install libraries and their **depencencies** (other libararies that the library you are installing needs).

**NOTE2:** If you were working on your computer you would have to provide the path where  is located the file you want to read. However since we are in a virtual location, we need to **mount** our Google Drive space in Colab. You can do it by importing the library google.colab  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#This syntax mean: from all the features that the library google.colab may have
#(and probably are heavy and may take a lot of time to load) please just load drive
#because is the only one we need.


**NOTE3**:
At the time you are running this, you will see an emergent window where Google Drive asks your permission for granting colab access to your Drive files. Please be sure that your browser allows pop-ups (emergent windows).

**NOTE4**:
With this permission I (and anybody) will not have access to your Drive. This line of code only mounts the Google Drive of the person running this cell.

**NOTE5**:
Now look at the left column of this Notebook, you will the directory *drive*. This directory contains all directories of your Google Drive. The root directory is *'/content/drive'*

## 2. Reading CVS files with Pandas

We are going to read the file **All_RTK_GPS_2024.csv**. This file contains the RTK GPS positions of the ground elevations at random points in the depression, and along just one of the GPR profiles

In [ ]:
#Load the file we are going to work with
#Personalize your own path. A Quick way is navigate through the left
#column, find the file, then right click and copy the path.
gps_file = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/All_RTK_GPS_2024.csv'
df = pd.read_csv(gps_file)

Let's play with the object df that means **data frame**, which is the name of the objects created when a file is read using pandas.

In [ ]:
#Let's see the columns or labels of our dataframe.
df.columns
#This information is important because those labels are the names
#to use for extracting the values of a given column.

In [ ]:
#We need to extract the coordinates and the elevation (Ellipsoidal height)
lat_gps = df['Latitude']
lat_utm_gps = df['Northing']
#This syntax mean: from the object df, extract the label whose header or label is 'Latitude'
print(lat_gps)
#The numbers before the actual value are a referece, but they are not part of the information extracted.

lon_gps = df['Longitude']
lon_utm_gps = df['Easting']

e_height_gps = df['Ellipsoidal height']

##3. Subroutines in Python

Since the elevation we extracted from the data frame is the ellipsoidal height and we need the geoidal height, the next step is to add 26.735m to **e_height**. Let's take advantange of that for introducing subroutines in Python.

Making a code in a single block could be inefficient and avoids testing specific parts of the algorithm. On the other hand, coding by using subroutines helps to make a more structured algorithm and, if some of the procedures have to be repeated, the subroutine can be call as many times as you need. Therefore there is no need to repeat the code.

The general structure of a subroutine is simple:

**Input -> Process -> Output**

You send some information (Input) that are used inside the subroutine (Process) and result is produced and sent to the main code (Output).

In [ ]:
#Let's to make a subroutine for converting ellipsoidal height to geoidal one
def ellipsoidal2geoidal(e_height):
    g_height = e_height + 26.735
    return(g_height)

#This syntax mean: define a subroutine named ellipsoidal2geoidal
#this subroutine will receive and object. Inside the subroutine create a new
#object named g_height that is equal to the object received plus 26.735.
#Finally return g_height to the main code.

In [ ]:
#Now call the subroutine
g_height_gps = ellipsoidal2geoidal(e_height_gps)
print(g_height_gps)

## 4. Reading Keyhole Markup Language (KML) files in Python

*   **KML**: It's an XML-based file format used for expressing geographic annotation and and visualization within internet-based mapping programs, such as Google Earth.
*   **KMZ**: KMZ stands for Keyhole Markup Language Zipped. It is a compressed version of the KML file format, typically used to package together multiple KML files, images, icons, and other resources into a single archive for distribution or sharing.

*Per se* we do not want to read KMZ files. We want to unzip them and read the KML files that are inside. For doing it  we need the libraries **ZIPFILE** for unzipping  and **GEOPANDAS** for reading the unzipped KML.

In [ ]:
import zipfile
import os

In [ ]:
#Path KMZ file
kmz_path = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/kmz_files_for_GPR_lines_a_cross_depression/MDLine02.kmz'
#Path to extract the KML files
extraction_dir = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/kmz_files_for_GPR_lines_a_cross_depression/Line02/'
#The path does not exist, so let's to create it using the os library for typing linux commands in Python
os.system('mkdir {}'.format(extraction_dir))

In [ ]:
#Once created the directory where we are going to extract the XML file, let's to unzip all
with zipfile.ZipFile(kmz_path, "r") as kmz:
  kmz.extractall(extraction_dir)
#Now check that in the recently created directory it is saved the file doc.kml.

In [ ]:
import geopandas as gpd
import fiona

In [ ]:
#Let's read the recently extracted doc.kml file
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gdf = gpd.read_file('{}/doc.kml'.format(extraction_dir),driver='KML')

#Let's to see its attributes
print(gdf)
print('****')
print(gdf.Name)
print('****')
print(gdf.Description)
print('****')
print(gdf.geometry)

##5. LineString and MultiLineString Geometries

Geometries of KML are diverse. In this case, the geometry is MultiLineString.
Unfortunately (as far as I know) it is not straightforward to access the coordinates of MultiLineString geometries but is quite easy for LineString ones.

Therefore, we need to convert the MultiLineString into a LineString. In this case, we can do this because although the geometry is named MultiLineString in the KML file, if you see any of the files in Google Earth they are simple lines.



In [ ]:
#The explode method converts a MultiLineString GeoDataFrame, into a LineString one
exploded = gdf.explode()
print(exploded)
print('*****')
#Now we can get the coordinates
coordinates = exploded.geometry.iloc[0].coords
print(coordinates)
print('*****')
#Coordinates are Python list that you can call element by element
print(coordinates[0])
print('*****')
#Each element contains 3 values corresponding to longitude, latitude, and elevation
print('Longitude first element')
print(coordinates[0][0])
print('Latitude first element')
print(coordinates[0][1])
print('Elevation first element')
print(coordinates[0][2])

print(len(coordinates[0]))

## 6. Repeating the same process for all the KMZ files
Repeat the same procedure for each point of each line in each KML file is unthinkably tedious and against the programming principles.

That is why we are going to make a subroutine for unzipping the KMZ files and reading the KML ones.

In the subroutine we will send:

*   The path of the KMZ file.
*   The name of the directory where we want to save the KML file. In order to preserve consistency, if the KMZ file is MDLine11, the directory for saving the doc.kml file extracted from it would be Line11.

At the end, the subroutine will return:

*   An **array** of coordinates and elevations

In [ ]:
def loadkmz(kmz_path,extraction_dir):
  #Subroutine for reading a KMZ file, unzipping it, reading a KML file and return the coordinates and the elevation.

  print('Reading {}'.format(kmz_path))
  #The path does not exist, so let's to create it using the os library for typing linux commands in Python
  print('mkdir {}'.format(extraction_dir))
  os.system('mkdir {}'.format(extraction_dir))

  #Once created the directory where we are going to extract the XML file, let's to unzip all
  with zipfile.ZipFile(kmz_path, "r") as kmz:
    kmz.extractall(extraction_dir)

  #Let's read the recently extracted doc.kml file
  gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
  gdf = gpd.read_file('{}/doc.kml'.format(extraction_dir),driver='KML')

  exploded = gdf.explode()
  #Now we can get the coordinates
  coordinates = exploded.geometry.iloc[0].coords

  lons = []
  lats = []
  elev = []

  for xyz in coordinates:
    for d in range(len(xyz)):
      if d == 0:
        lons.append(xyz[d])
      elif d == 1:
        lats.append(xyz[d])
      elif d == 2:
        elev.append(xyz[d])

  print('Done it!')
  return(lons,lats,elev)

In [ ]:
#Path KMZ file
kmz_path = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/kmz_files_for_GPR_lines_a_cross_depression/MDLine05.kmz'
#Path to extract the KML files
extraction_dir = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/kmz_files_for_GPR_lines_a_cross_depression/Line05/'
lons5,lats5,elev5 = loadkmz(kmz_path,extraction_dir)

In [ ]:
print(lons5)
print(lats5)
print(elev5)

## 7. Convert Geographic coordinates to Universal Transverse Mercator (UTM) system

We need to solve a last step. The coordinates are given in pairs of latitute and longitude and we need them in UTM.

For doing this, we need the library pyproj and again, to make a subroutine where input is a longitude, latitude pair and the UTM zone we are going to use as a reference. In return, the subroutine will provide a x,y point in UTM.



In [ ]:
import pyproj

In [ ]:
def geo_to_utm(lons,lats,utm_zone,show):

    p = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')
    h=[]
    v=[]

    for i in range(len(lons)):
        x,y = p(lons[i],lats[i])
        h.append(x)
        v.append(y)

    if show:
      for i in range(len(lons)):
          print('GEO: {} {}'.format(lons[i],lats[i]))
          print('UTM (zone {}): {} {}'.format(utm_zone,h[i],v[i]))
          print('****')

    return(h,v)

In [ ]:
#Let's make the conversion
utm_zone = 17
show = True
x5,y5 = geo_to_utm(lons5,lats5,utm_zone,show)

Now, let's to wrap-up and repear the procedure for all the KMZ files

**NOTE**: The code could be more efficient if we only give the directory where all the kmz files are. This would be an interesting homework.

In the meantime, let's call the subtroutines seven times, one per line.

In [ ]:
utm_zone = 17
show = False
main_dir = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/kmz_files_for_GPR_lines_a_cross_depression'

#Line2
kmz_path = '{}/MDLine02.kmz'.format(main_dir)
extraction_dir = '{}/Line02/'.format(main_dir)
lons2,lats2,elev2 = loadkmz(kmz_path,extraction_dir)
x2,y2 = geo_to_utm(lons2,lats2,utm_zone,show)

#Line5
kmz_path = '{}/MDLine05.kmz'.format(main_dir)
extraction_dir = '{}/Line05/'.format(main_dir)
lons5,lats5,elev5 = loadkmz(kmz_path,extraction_dir)
x5,y5 = geo_to_utm(lons5,lats5,utm_zone,show)

#Line7
kmz_path = '{}/MDLine07.kmz'.format(main_dir)
extraction_dir = '{}/Line07/'.format(main_dir)
lons7,lats7,elev7 = loadkmz(kmz_path,extraction_dir)
x7,y7 = geo_to_utm(lons7,lats7,utm_zone,show)

#Line8
kmz_path = '{}/MDLine08.kmz'.format(main_dir)
extraction_dir = '{}/Line08/'.format(main_dir)
lons8,lats8,elev8 = loadkmz(kmz_path,extraction_dir)
x8,y8 = geo_to_utm(lons8,lats8,utm_zone,show)

#Line9
kmz_path = '{}/MDLine09.kmz'.format(main_dir)
extraction_dir = '{}/Line09/'.format(main_dir)
lons9,lats9,elev9 = loadkmz(kmz_path,extraction_dir)
x9,y9 = geo_to_utm(lons9,lats9,utm_zone,show)

#Line10
kmz_path = '{}/MDLine10.kmz'.format(main_dir)
extraction_dir = '{}/Line10/'.format(main_dir)
lons10,lats10,elev10 = loadkmz(kmz_path,extraction_dir)
x10,y10 = geo_to_utm(lons10,lats10,utm_zone,show)

#Line11
kmz_path = '{}/MDLine11.kmz'.format(main_dir)
extraction_dir = '{}/Line11/'.format(main_dir)
lons11,lats11,elev11 = loadkmz(kmz_path,extraction_dir)
x11,y11 = geo_to_utm(lons11,lats11,utm_zone,show)

In [ ]:
#Gps data conversion
xGPS,yGPS = geo_to_utm(lon_gps,lat_gps,utm_zone,show)

## 8. Plotting the data

For observing the GPR measurements and the GPS data we are going to use the
matplotlib library (and also numpy). So we are going to import them and make another subroutine for plotting the data. That subroutine will receive the UTM coordinates and the elevation and in exchange, it will return the plot we want.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
def plot_line_profile(x,y,z,profile,path_save):
  # Generate sample data
  x = np.array(x)
  y = np.array(y)
  z = np.array(z)

  # Create a scatter plot with Z values represented by color
  plt.figure(figsize=(8, 6))
  plt.scatter(x, y, c=z, cmap='viridis', s=100)  # s controls the size of the dots
  plt.colorbar(label='Elevation (m)')  # Add color bar to show Z values
  plt.xlabel('Easting')
  plt.ylabel('Northing')
  plt.title('Elevation along profile {}'.format(profile))
  plt.grid(True)
  plt.savefig('{}/scatter_plot_profile_{}.pdf'.format(path_save,profile))
  files.download('{}/scatter_plot_profile_{}.pdf'.format(path_save,profile))
  plt.show()


In [ ]:
#We need to make a directory for saving the figures
path_save = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/Figures'
os.system('mkdir {}'.format(path_save))
#Let's define the profile number we want to plot for saving the figure with the appropiate name.

profile = '9'
plot_line_profile(x9,y9,elev9,profile,path_save)

In [ ]:
#We can plot also the RTK GPS positions
path_save = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/Figures'
profile = 'GPS'
plot_line_profile(xGPS,yGPS,g_height_gps,profile,path_save)

In [ ]:
#Now let's plot all the profiles
all_x = x2+x5+x7+x8+x9+x10+x11
all_y = y2+y5+y7+y8+y9+y10+y11
all_elevations = elev2+elev5+elev7+elev8+elev9+elev10+elev11

In [ ]:
profile = 'all'
path_save = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/Figures'
plot_line_profile(all_x,all_y,all_elevations,profile,path_save)

## 9. Contour Map

The last step is making a subroutine that takes all the profiles from the GPR measurmements and interpolates a continuous surface

In [ ]:
def make_contour_map(all_x,all_y,all_elevations,path_save):
  plt.figure(figsize=(8, 6))
  plt.gca().set_aspect("equal")
  cp = plt.tricontourf(all_x,all_y,all_elevations, cmap = 'viridis')
  plt.xlabel('Easting')
  plt.ylabel('Northing')
  cbar = plt.colorbar(cp)
  cbar.set_label('Elevation (m)', rotation = 270, labelpad = 10)
  plt.savefig('{}/contour_map.pdf'.format(path_save,profile))
  files.download('{}/contour_map.pdf'.format(path_save,profile))
  plt.show()

In [ ]:
path_save = '/content/drive/MyDrive/Class_Dr_Kruse/clean_notebooks/Figures'
make_contour_map(all_x,all_y,all_elevations,path_save)

This the end so far. You can modify many parameters for making the interpolations and hence the contour maps.